<a href="https://colab.research.google.com/github/sujeongEOM/python-study/blob/main/encryption.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pycryptodome

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

reference
https://pycryptodome.readthedocs.io/en/latest/src/examples.html

In [ ]:
#Generate public key and private key

from Crypto.PublicKey import RSA

key = RSA.generate(2048)
private_key = key.export_key()
file_out = open("private.pem", "wb")
file_out.write(private_key)
file_out.close()

public_key = key.publickey().export_key()
file_out = open("receiver.pem", "wb")
file_out.write(public_key)
file_out.close()

In [ ]:
#Encrypt data with RSA

from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
from Crypto.Cipher import AES, PKCS1_OAEP
import pandas as pd

#구글드라이브에서 암호화하려는 csv 파일 가져오기
df = pd.read_csv("/content/drive/MyDrive/chestpa_BMD_20210414DXA.xls - Sheet1.csv")

#시리즈에서 리스트로 변환
pt_nb = df['실명등록번호']
pt_nb_list = pt_nb.values.tolist()
final_list = []
encrypted_list = []

#리스트 요소 dtype=str, 암호화는 byte로 이루어지기에 byte로 변환
for i in pt_nb_list:
    final_data = i.encode()
    final_list.append(final_data)

for y in final_list:
    file_out = open("encrypted_data.bin", "wb")
    recipient_key = RSA.import_key(open("receiver.pem").read())
    session_key = get_random_bytes(16)

# Encrypt the session key with the public RSA key
    cipher_rsa = PKCS1_OAEP.new(recipient_key)
    enc_session_key = cipher_rsa.encrypt(session_key)

# Encrypt the data with the AES session key
    cipher_aes = AES.new(session_key, AES.MODE_EAX)
    ciphertext, tag = cipher_aes.encrypt_and_digest(y)
    [ file_out.write(x) for x in (enc_session_key, cipher_aes.nonce, tag, ciphertext) ]
    encrypted_list.append(ciphertext)
    file_out.close()

encrypted_df = df.copy()
encrypted_df['실명등록번호'] = encrypted_list

encrypted_df.to_csv("/content/drive/MyDrive/Colab Notebooks/chestpa_BMD_20210414DXA_encrypted.csv", mode='w')